# AutoKeras

**This code worked with the AutoKeras version in Nov 2019 and has since been depreciated. Please refer to the June 2020 version [code/chapter-5/5-autokeras.ipynb](https://github.com/PracticalDL/Practical-Deep-Learning-Book/blob/master/code/chapter-5/5-autokeras.ipynb)** 

As AI is automating more and more jobs, it can finally automate designing AI architectures too. Neural Architecture Search (NAS) approaches utilize reinforcement learning to join together mini architectural blocks, till they are able to maximize the objective function - i.e. our validation accuracy. The current state of the art networks are all based on NAS, leaving human-designed architectures in the dust. Research in this area started showing promising results in 2017, with a bigger focus on making train faster in 2018. 

AutoKeras (Haifeng Jin et al), also apply this state of the art technique on our particular datasets in a relatively accessible manner. Generating new model architectures with AutoKeras is a matter of supplying our images and associated labels, and a time limit to finish running the jobs by. Internally, it implements several optimization algorithms including a Bayesian optimization approach to search for an optimal architecture. 


In [ ]:
# Make sure you have the necessary packages installed
!pip3 install autokeras
!pip3 install graphviz

In [ ]:
from keras.datasets import mnist
from autokeras.image.image_supervised import ImageClassifier

In [ ]:
# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape + (1, ))
x_test = x_test.reshape(x_test.shape + (1, ))

Define an image classifier that performs augmentations

In [ ]:
clf = ImageClassifier(path=".",verbose=True, augment=True, \
                     searcher_args={'trainer_args':{'max_iter_num':7}}
                                    )

Fit the classifier within a time limit of 60 seconds * 60 minutes * 1 hour 

In [ ]:
clf.fit(x_train, y_train, time_limit=60 * 60 * 1)
clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)

In [ ]:
y = clf.evaluate(x_test, y_test)

In [ ]:
print(y)

This is what the complete output looks like: 

```
Saving Directory: .
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                               
Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   1.6381531059741974   |   0.8960000000000001   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+
Epoch-1, Current Metric - 0:  43%|████████████                | 200/465 [33:54<46:26, 10.52s/ batch]Time is out.
                                                                  
No loss decrease after 30 epochs.

0.9852
```

Note that early stopping is present as well.

In [ ]:
clf.export_autokeras_model("model.pkl")

Let's visualize the trained model.

In [ ]:
visualize('.')

One can also use `scikit-learn` to verify accuracy as follows:

In [ ]:
from sklearn.metrics import accuracy_score
y = clf.predict(x_test)
accuracy_score(y_true=y_test, y_pred=y)